In [1]:
import ipywidgets as widgets
from autocalc.autocalc import Var
from PIL import Image
import requests
import matplotlib.pyplot as plt
import json
import urllib.parse as urlparse
from urllib.parse import urlencode
import re
from IPython.display import Image
from IPython.display import display, clear_output

city = Var('city:',initial_value = 'london', widget = widgets.Text())
display(city)


In [ ]:
button_send=widgets.Button(description='enter',tooltip='Send',style={'description_width':'initial'})


output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()
        
        parameters_0 = {
            "q": city._v,
            "appid": "d8aaad5f4b47f70adc98da3406b9021c"
        }

        response_loc = requests.get("http://api.openweathermap.org/geo/1.0/direct", params=parameters_0)

        parameters = {
            "lon": response_loc.json()[0]['lon'],
            "lat": response_loc.json()[0]['lat'],
            "appid": "d8aaad5f4b47f70adc98da3406b9021c"
        }

        response_for = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)
        response_curr = requests.get("https://api.openweathermap.org/data/2.5/weather", params=parameters)
        response_poll = requests.get("http://api.openweathermap.org/data/2.5/air_pollution", params=parameters)

        print("current weather:\n")
        icon=response_curr.json()['weather'][0]['icon']
        path="".join([icon, "@2x.png"])
        url="".join(["http://openweathermap.org/img/wn/", path])
        display(Image(url=url))
        
        print("current temperature:",'{:0.0f}'.format(response_curr.json()['main']['temp']-273.15),"C")
        print("feels like:",'{:0.0f}'.format(response_curr.json()['main']['feels_like']-273.15),"C")
        print("minimum temperature:",'{:0.0f}'.format(response_curr.json()['main']['temp_min']-273.15),"C")
        print("maximum temperature:",'{:0.0f}'.format(response_curr.json()['main']['temp_max']-273.15),"C")
        print("weather condition:",response_curr.json()['weather'][0]['main'])

        def numbers_to_strings(argument):
            switcher = {
                1: "good",
                2: "satisfactory",
                3: "moderate",
                4: "poor",
                5: "very poor",
            }
            return switcher.get(argument, "nothing")

        
        print("air condition:",numbers_to_strings(response_poll.json()['list'][0]['main']['aqi']))
        print("\nweather forecast:\n")

        #times_pattern = r"^(\d{4})-(\d{2})-(?P<day>\d{2}) (?P<hour>\d{2}):\d{2}:\d{2}"
        for var in response_for.json()['list']:
           #matches = re.search(times_pattern,var['dt_txt'])
           #time = matches.group('hour')
           timing=var.get('dt_txt')
           print(timing)
           temp=var.get('main').get('temp')
           print('{:0.0f}'.format(temp-273.15),"C")
           a=var.get('weather')[0]['icon']
           path="".join([a, "@2x.png"])
           url="".join(["http://openweathermap.org/img/wn/", path])
           display(Image(url=url))

    

        
button_send.on_click(on_button_clicked)
vbox_result = widgets.HBox([button_send, output])
display(vbox_result)
